# Using R in python thanks to rpy2

Python and R are two widely used programing languages. You may want to run some R code in python for example to use a R-package or to integrate R-function. 
To integrate some R code in python, you need to have R install on your machine and to install the python package rpy2 (pip install rpy2). 

Documentation :
https://rpy2.github.io 

## Starting point

In [2]:
import rpy2.robjects as ro

## Importation of packages

You can import any package by using importr and by quoting the name of the package

In [3]:
from rpy2.robjects.packages import importr
importr("base")



rpy2.robjects.packages.Package as a <module 'base'>

## Definition of parameters in R

To define parameters in the R environment use 
ro.r(''' definition of parameter  '''). What's written between ' ' ' is R code like it would be in R.

In [4]:
ro.r(''' value <- 50''')
ro.r('value')

50.000000


In [5]:
ro.r('''string <- "message"''')
ro.r('string')

'message'


## Conversion of python parameter in R

Parameters given to an R function should be R objects. So a conversion is needed to pass from python object to R object. Here are some examples of conversion:

### List of integers

In [6]:
intListPython = [1, 2, 3, 4, 5, 6]

In [7]:
intVectorR = ro.IntVector(intListPython) 
print(intVectorR, type(intVectorR))

[1] 1 2 3 4 5 6
 <class 'rpy2.robjects.vectors.IntVector'>


### List of float

In [8]:
floatListPython = [1.6, 2.5, 3.3, 4.5]

In [9]:
floatVectorR = ro.FloatVector(floatListPython)
print(floatVectorR, type(floatVectorR))

[1] 1.6 2.5 3.3 4.5
 <class 'rpy2.robjects.vectors.FloatVector'>


### List of string

In [10]:
stringListPyhton = ["a", "b", "c", "d"]

In [11]:
stringListR = ro.StrVector(stringListPyhton)
print(stringListR, type(stringListR))

[1] "a" "b" "c" "d"
 <class 'rpy2.robjects.vectors.StrVector'>


## Using R function

### Function definition

The function needs to be define first in the R environnement :

In [12]:
ro.r('''function_in_R <- function(){
    data(iris)
    sepal_mean = mean(iris$Sepal.Length)
    return(sepal_mean)}
    ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000002B7D59147C8> [RTYPES.CLOSXP]
R classes: ('function',)

Then to be able to call the function in python, you need to define the R function in python.

In [13]:
r_function_in_python = ro.r('function_in_R')
sepal_mean = r_function_in_python()
print(sepal_mean)

[1] 5.843333



### Example with numbers as argument

In [14]:
ro.r('''opertation_in_R <- function(a, b, c){
    d <- a*b+c
    return(d)}
    ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000002B7D4DE2A08> [RTYPES.CLOSXP]
R classes: ('function',)

In [15]:
r_function_in_python = ro.r('opertation_in_R')
d = r_function_in_python(3, 5, 10)
print(d)

[1] 25



### Example with a list as argument

In [16]:
ro.r('''stat_in_R <- function(list_value){
    sum_value <- sum(list_value)
    mean_value <- mean(list_value)
    var_value <- var(list_value)
    test <- c(sum_value, mean_value,var_value)
    return(test)}
    ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000002B7D58FB388> [RTYPES.CLOSXP]
R classes: ('function',)

In [17]:
r_function_in_python = ro.r('stat_in_R')
param = ro.FloatVector([3.3, 3.4, 3.5, 3.3, 3.4, 3.6, 3.3, 3.2, 3.3])
d = r_function_in_python(param)
print(d)
print(d[0], type(d[0]), d[0]*9) #how to use one of the results

[1] 30.300000  3.366667  0.015000

30.3 <class 'float'> 272.7


## Using DataFrame 

DataFrame is a frequent structure used in R and in python. A conversion is needed to pass from pandas dataFrame to R dataFrame.

### From pandas to R

In [18]:
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [19]:
pd_DataFrame = pd.DataFrame({ "val1": [11, 22, 33, 44, 55],
                            "val2": [44, 53, 56, 45, 66] })

with localconverter(ro.default_converter + pandas2ri.converter):
    r_DataFrame = ro.conversion.py2rpy(pd_DataFrame)

print(pd_DataFrame, "\n",type(pd_DataFrame))
print(r_DataFrame, type(r_DataFrame))

   val1  val2
0    11    44
1    22    53
2    33    56
3    44    45
4    55    66 
 <class 'pandas.core.frame.DataFrame'>
  val1 val2
0   11   44
1   22   53
2   33   56
3   44   45
4   55   66
 <class 'rpy2.robjects.vectors.DataFrame'>


### From R to pandas

In [20]:
ro.r('''
 data = data.frame( val1 = c(11, 22, 33, 44), val2 = c(12, 23, 34, 45))
''')
r_DataFrame = ro.r('data')
print(r_DataFrame)

with localconverter(ro.default_converter + pandas2ri.converter):
    pd_DataFrame = ro.conversion.rpy2py(r_DataFrame)

print(pd_DataFrame)

  val1 val2
1   11   12
2   22   23
3   33   34
4   44   45

   val1  val2
1  11.0  12.0
2  22.0  23.0
3  33.0  34.0
4  44.0  45.0


Careful as you can see the rank id is not the same if you convert from python to R or from R to python.